# **_Importation des produits_**

In [ ]:
#Importation des bibliothèques
import requests 
import asyncio
import aiohttp
import pandas as pd
from tqdm.asyncio import tqdm

size=50000 #nombre d'élément que l'on souhaite par page

#URL pour obtenir la dernière page
lastPageUrl="https://prices.openfoodfacts.org/api/v1/products?size={}"

#obtention de la dernière page
try:
    responseLastPage = requests.get(lastPageUrl.format(size)) #interrogation de la l'API Factfood pour obtenir la dernière page
    jsonLastPage=responseLastPage.json() #transformation en json
    lastPage=jsonLastPage['pages']
    #print(lastPage)
except Exception as e:
    print(f"Une erreur s'est produite lors de l'obtention de la dernière page: {e}")

In [ ]:
#Endpoint pour les produits
productsUrl = "https://prices.openfoodfacts.org/api/v1/products?page={}&size={}"
productList = []  # Liste globale pour stocker les produits collectés

async def fetch_page(session, page, size):
    """Récupère les produits pour une page avec gestion des erreurs et des répétitions."""
    try:
        async with session.get(productsUrl.format(page, size)) as response:
            # Vérifie que la réponse est bien JSON
            if response.status == 200:
                return await response.json()
            else:
                print(f"Page {page} - Erreur HTTP {response.status}")
    except Exception as e:
        print(f"Page {page} - erreur : {e}")
    return None

async def getProducts(lastPage, size=size):
    """Récupère les produits jusqu'à la dernière page avec gestion des erreurs."""
    async with aiohttp.ClientSession() as session:
        for page in tqdm(range(1, lastPage + 1), desc="Collecte des produits"):
            jsonProduct = await fetch_page(session, page, size)
            if jsonProduct and "items" in jsonProduct:
                productList.extend(jsonProduct["items"])  # Ajoute les produits collectés
            else:
                print(f"Page {page} - Aucune donnée collectée")
    print(f"Total de produits collectés : {len(productList)}")


In [ ]:
await getProducts(lastPage=lastPage)

In [ ]:
dfProduct=pd.json_normalize(productList)
dfProductSpark=spark.createDataFrame(dfProduct)

In [ ]:
dfProductSpark.write.format("parquet").mode("overwrite").save("Files/Bronze/products.parquet")